# Flower classification

Here i'd like to present u a simple flower classification based on leaf and stem options


In [33]:
import tensorflow as tf
import pandas as pd

In [2]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [29]:
dtrain = pd.read_csv("https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv", names=CSV_COLUMN_NAMES, header=0)
dtest = pd.read_csv("https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv", names=CSV_COLUMN_NAMES, header=0)

In [22]:
dtrain.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [30]:
y_train = dtrain.pop('Species')
y_test = dtest.pop('Species')

In [39]:
def input_function(features, answers, training=True, batch_size=256):
    ds = tf.data.Dataset.from_tensor_slices((dict(features), answers))
    
    if training:
        ds = ds.shuffle(1000).repeat()
        
    return ds.batch(batch_size)    

In [42]:
feature_cols = []
for key in dtrain.keys():
    feature_cols.append(tf.feature_column.numeric_column(key=key))

In [43]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_cols,
    hidden_units=[30, 10],
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpgxw1n1mt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [44]:
classifier.train(
    lambda: input_function(dtrain, y_train),
    steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpgxw1n1mt/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.3370606, step = 0
INFO:tensorflow:global_step/sec: 525.736
INFO:tensorflow:loss = 1.0634687, step = 100 (0.191 sec)
INFO:tensorflow:global_step/sec: 692.2
INFO:tensorflow:loss = 0.96238923, step = 200 (0.145 sec)
INFO:tensorflow:global_step/sec:

In [45]:
result = classifier.evaluate(lambda: input_function(dtest, y_test, training=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-08T18:28:45Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpgxw1n1mt/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.62784s
INFO:tensorflow:Finished evaluation at 2021-01-08-18:28:45
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.96666664, average_loss = 0.4103121, global_step = 5000, loss = 0.4103121
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpgxw1n1mt/model.ckpt-5000


In [50]:
result['accuracy']

0.96666664

# Playing with out model

Here i've written some code so that i can predict the flower class based on input parameters

In [52]:
def input_f(data, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(data)).batch(batch_size=batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

In [78]:
print("Please type numeric values as prompted")
for feature in features:
    val = input(feature + ": ")
    predict[feature] = [float(val)]
    
predictions = classifier.predict(lambda: input_f(predict))

for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))

Please type numeric values as prompted
SepalLength: 21
SepalWidth: 21
PetalLength: 21
PetalWidth: 21
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpgxw1n1mt/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Virginica" (99.6%)
